# Plot some first SPECS Calculations for Eriswil


In [11]:
%matplotlib inline
%matplotlib widget

In [12]:
import glob
import numpy as np
import xarray as xr

import thermodynamic_variables as td

In [34]:
from typing import Dict
import copy
import multiprocessing
import concurrent.futures
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import re
from ipywidgets import interact, widgets, fixed

HHLd = np.array(
    [21.500002  , 20.514286  , 19.556965  , 18.62768   , 17.72607   ,
    16.851786  , 16.004465  , 15.183751  , 14.389286  , 13.620715  ,
    12.87768   , 12.159819  , 11.474235  , 10.83433   , 10.231516  ,
    9.650918  ,  9.092202  ,  8.555033  ,  8.039072  ,  7.5439944 ,
    7.069464  ,  6.615141  ,  6.180692  ,  5.76579   ,  5.370097  ,
    4.993273  ,  4.634993  ,  4.294922  ,  3.9727197 ,  3.6680555 ,
    3.3805976 ,  3.110009  ,  2.855953  ,  2.6181023 ,  2.3961203 ,
    2.18967   ,  1.9984195 ,  1.8220367 ,  1.6601849 ,  1.5125275 ,
    1.3787366 ,  1.2584761 ,  1.151409  ,  1.0572032 ,  0.9755266 ,
    0.9060427 ,  0.84841454,  0.8023148 ,  0.76740676,  0.7433537 ]
    )

RGRENZ = np.array(
    [1.25992106e-09, 1.58740110e-09, 1.99999994e-09, 2.51984211e-09,
    3.17480220e-09, 3.99999989e-09, 5.03968423e-09, 6.34960440e-09,
    7.99999977e-09, 1.00793685e-08, 1.26992088e-08, 1.59999995e-08,
    2.01587369e-08, 2.53984176e-08, 3.19999991e-08, 4.03174738e-08,
    5.07968352e-08, 6.39999982e-08, 8.06349476e-08, 1.01593670e-07,
    1.27999996e-07, 1.61269895e-07, 2.03187341e-07, 2.55999993e-07,
    3.22539790e-07, 4.06374681e-07, 5.11999986e-07, 6.45079581e-07,
    8.12749363e-07, 1.02399997e-06, 1.29015916e-06, 1.62549873e-06,
    2.04799994e-06, 2.58031832e-06, 3.25099745e-06, 4.09599988e-06,
    5.16063665e-06, 6.50199490e-06, 8.19199977e-06, 1.03212733e-05,
    1.30039898e-05, 1.63839995e-05, 2.06425466e-05, 2.60079796e-05,
    3.27679991e-05, 4.12850932e-05, 5.20159592e-05, 6.55359981e-05,
    8.25701864e-05, 1.04031918e-04, 1.31071996e-04, 1.65140373e-04,
    2.08063837e-04, 2.62143993e-04, 3.30280745e-04, 4.16127674e-04,
    5.24287985e-04, 6.60561491e-04, 8.32255348e-04, 1.04857597e-03,
    1.32112298e-03, 1.66451070e-03, 2.09715194e-03, 2.64224596e-03,
    3.32902139e-03, 4.19430388e-03]
)

def format_string(s: str) -> str:
    """Formats input string to a certain format."""
    return "INP = {}, FLARE = {}".format(*re.findall(r'[0-9\.e]+', s), 'FEno' if 'FEno' in s else '') 



class MultiPanelPlot:
    def __init__(

        self, 
        datasets: Dict[str, xr.Dataset], 
        varname: str, 
        mode: str,         
        timestep0: int | list = None,
        timeframe: str = 'single', 
        vmin: float = 1.0,      vmax: float = 1.0e4, 
        nrows: int = 2,         ncols: int = 3, 
        hmin: float = 42,       hmax: float = 44, 
        xmin: float = 1.0e-9,   xmax: float = 1.0e-2, 
        ymin: float = 0.0,      ymax: float = 12.0, 

    ) -> None:
        """
        Initializer for the MultiPanelPlot class. 

        Args:
            datasets (dict of str: xr.Dataset): The dictionary of data sets for plotting.
            varname (str): The variable name in the data sets.
            mode (str): The plotting mode ('profile' or 'area').
            vmin (float, optional): The minimum value for the color bar. Defaults to 1.0.
            vmax (float, optional): The maximum value for the color bar. Defaults to 1.0e4.
            nrows (int, optional): The number of rows in the subplot. Defaults to 2.
            ncols (int, optional): The number of columns in the subplot. Defaults to 3.
            hmin (float, optional): The minimum model level height. Defaults to 42.
            hmax (float, optional): The maximum model level height. Defaults to 44.
            timeframe (str, optional): The timeframe to plot ('single' or other). Defaults to 'single'.
            xmin (float, optional): The minimum x-value. Defaults to 1.0e-9.
            xmax (float, optional): The maximum x-value. Defaults to 1.0e-2.
            ymin (float, optional): The minimum y-value. Defaults to 0.0.
            ymax (float, optional): The maximum y-value. Defaults to 12.0.
            timestep0 (int or list, optional): The time step for the plot. Defaults to 0 or [0, 10].
        """
        
        self.datasets = copy.deepcopy(datasets)
        self.varname =  varname
        self.mode = mode
        key = next(iter(datasets))
        self.time = datasets[key].time.values
        self.x = datasets[key].x.values
        self.y = datasets[key].y.values
        self.height = HHLd
        self.xlim = (xmin, xmax)
        self.ylim = (ymin, ymax)
        self.hlim = (hmin, hmax)
        self.timeframe = timeframe
        self.bins = np.arange(datasets[key][varname].shape[-1])
        self.rgrenz = RGRENZ
        self.standard_name, self.unit = datasets[key][varname].attrs.values()
        self.cmap = mpl.colors.LinearSegmentedColormap.from_list(
            'mycmap', [
                (0, 'white'), (0.2, 'blue'), (0.4, 'cyan'), (0.5, 'lime'), 
                (0.6, 'yellow'), (0.9, 'red'), (1, 'purple')
                ]
            ) 
        self.norm = colors.LogNorm(vmin, vmax)
        self.n_axes = nrows * ncols

        mpl.rcParams['font.size'] = 10
        self.pmeshs = []
        self.fig, self.axes = plt.subplots(nrows, ncols, figsize=(ncols*5,nrows*3.33), constrained_layout=True)
        if nrows*ncols==1:
            self.axes = np.array(self.axes)
        self.fig.subplots_adjust(hspace=0.3, wspace=0.15, left=0.15, right=0.8, top=0.85, bottom=0.15)  # make space for colorbar
        self.timestep = 0 if (timestep0 is None and timeframe == 'single') else (timestep0 if timestep0 is not None else [0, 10])
        print(self.timestep)
        self.init_first_plot(self.timestep)


    def set_colorbar(self) -> None:
        """
        Creates and sets a color bar for the figure.
        """
        cbar_ax = self.fig.add_axes([0.89, 0.21, 0.01, 0.6])  # adjust these values as needed
        cbar = self.fig.colorbar(
            cm.ScalarMappable(norm=self.norm, cmap=self.cmap), 
            cax=cbar_ax, 
            orientation='vertical', 
            extend='both', 
            )
        cbar.set_label(self.unit)
        self.colorbar = cbar


    def set_name_tick_params(self, ax: mpl.axes.Axes, name: str) -> None:
        """
        Sets the title and tick parameters for an axes.

        Args:
            ax (mpl.axes.Axes): The axes to configure.
            name (str): The title for the axes.
        """
        #ax.set_title(format_string(name))
        ax.tick_params(which='both', direction='in')
        ax.minorticks_on()
        ax.grid(True, which='major', linestyle='-', linewidth='0.5', color='black', alpha=0.5)
        ax.grid(True, which='minor', linestyle=':', linewidth='0.5', color='black', alpha=0.25)


    def init_first_plot(self, timestep: int) -> None:
        """
        Initializes the first plot according to the plot mode.

        Args:
            timestep (int): The time step for the plot.
        """
        if self.mode == 'profile':
            self._init_profile_plot(timestep)
        elif self.mode == 'timeseries':
            self._init_timeseries_plot(timestep)         
        elif self.mode == 'area':
            self._init_area_plot(timestep)
        else:
            raise ValueError('Wrong mode given. Available: "profile", "timeseries", "area"')

    def _init_profile_plot(self, timestep: int) -> None:
        """
        Initializes the profile plot.

        Args:
            timestep (int): The time step for the plot.
        """
        for (name, ds), ax in zip(self.datasets.items(), self.axes.flat):
            data = ds[self.varname][timestep, :, 12, 12, :]
            pcm = ax.pcolormesh(self.rgrenz, self.height, data, norm=self.norm, cmap=self.cmap)
            self.pmeshs.append(pcm)
            ax.set(ylim=self.ylim, xlim=self.xlim, xscale='log')
            self.set_name_tick_params(ax, name)
        
        self.fig.text(0.5, 0.06, 'radius [m]', ha='center', va='center')
        self.fig.text(0.1, 0.5, 'height [km]', ha='center', va='center', rotation='vertical')
        self.set_colorbar()            


    def _init_timeseries_plot(self, timestep: int) -> None:
        """
        Initializes the timeseries plot.

        Args:
            timestep (int): The time step for the plot.
        """
        for (name, ds), ax in zip(self.datasets.items(), self.axes.flat):
            data = ds[self.varname][:, :, 12, 12, :].sum(dim='bin').T
            pcm = ax.pcolormesh(self.time, self.height, data, norm=self.norm, cmap=self.cmap)
            self.pmeshs.append(pcm)
            ax.set(ylim=self.ylim)
            self.set_name_tick_params(ax, name)
        
        self.fig.text(0.5, 0.06, 'time [step]', ha='center', va='center')
        self.fig.text(0.1, 0.5, 'height [km]', ha='center', va='center', rotation='vertical')
        self.set_colorbar()


    def _init_area_plot(self, timestep: int | list) -> None:
        """
        Initializes the area plot.

        Args:
            timestep (int): The time step for the plot.
        """
        for (name, ds), ax in zip(self.datasets.items(), self.axes.flat):
            data = self._get_data_area(ds, timestep, self.hlim[0], self.hlim[1])
            pcm = ax.pcolormesh(self.x, self.y, data, norm=self.norm, cmap=self.cmap)
            self.pmeshs.append(pcm)
            self.set_name_tick_params(ax, name)
        
        self.fig.text(0.5, 0.06, 'x [-]', ha='center', va='center')
        self.fig.text(0.1, 0.5, 'y [-]', ha='center', va='center', rotation='vertical')
        self.set_colorbar()


    def _get_data_area(self, ds: xr.Dataset, timestep: int | list, hmin: float, hmax: float) -> xr.DataArray:
        """
        Gets the data for the area plot.

        Args:
            ds (xr.Dataset): The data set.
            timestep (int): The time step.
            hmin (float): The minimum height.
            hmax (float): The maximum height.

        Returns:
            xr.DataArray: The data for the plot.
        """
        if self.timeframe == 'single':
            # Sum over the last two dimensions
            return ds[self.varname][timestep, hmin:hmax, :, :, :].sum(dim=('z', 'bin'))
        elif self.timeframe == 'interval':
            # Sum over timeframe and the last two dimensions
            return ds[self.varname][timestep[0]:timestep[-1], hmin:hmax, :, :, :].sum(dim=('time', 'z', 'bin'))
        elif self.timeframe == 'all':
            # Sum over time and the last two dimensions
            return ds[self.varname][   :    , hmin:hmax, :, :, :].sum(dim=('time', 'z', 'bin'))


    def display(self, timestep: int = 0, x: int = 12, y: int = 12, hmin: float = 42, hmax: float = 44) -> None:
        """
        Displays the plot according to the plot mode.

        Args:
            timestep (int, optional): The time step for the plot. Defaults to 0.
            x (int, optional): The x-value for the plot. Defaults to 12.
            y (int, optional): The y-value for the plot. Defaults to 12.
            hmin (float, optional): The minimum height. Defaults to 42.
            hmax (float, optional): The maximum height. Defaults to 44.
        """
        if self.mode == 'profile':
            self._display_profile(timestep=timestep, x=x, y=y)
        elif self.mode =='timeseries':
            self._display_timeseries(x=x, y=y)    
        elif self.mode == 'area':
            self._display_area(timestep=timestep, hmin=hmin, hmax=hmax)

        plt.show()


    def _display_profile(self, timestep: int = 0, x: int = 12, y: int = 12) -> None:
        """
        Displays the profile plot.

        Args:
            timestep (int, optional): The time step for the plot. Defaults to 0.
            x (int, optional): The x-value for the plot. Defaults to 12.
            y (int, optional): The y-value for the plot. Defaults to 12.
        """
        for i, (name, ds) in enumerate(self.datasets.items()):
            if i < self.n_axes and self.varname in ds:
                data = ds[self.varname][timestep, :, y, x, :]
                self.pmeshs[i].set_array(data.values.ravel())


    def _display_area(self, timestep: int | list, hmin: float, hmax: float) -> None:
        """
        Displays the area plot.

        Args:
            timestep (int): The time step for the plot.
            hmin (float): The minimum height.
            hmax (float): The maximum height.
        """
        for i, (name, ds) in enumerate(self.datasets.items()):
            if i < self.n_axes and self.varname in ds:
                data = self._get_data_area(ds, timestep, hmin, hmax)
                self.pmeshs[i].set_array(data.values.ravel())   


    def _display_timeseries(self, x: int = 12, y: int = 12) -> None:
        """
        Displays the timeseries plot.

        Args:
            timestep (int): The time step for the plot.
            hmin (float): The minimum height.
            hmax (float): The maximum height.
        """
        for i, (name, ds) in enumerate(self.datasets.items()):
            if i < self.n_axes and self.varname in ds:
                data = ds[self.varname][:, :, y, x, :].sum(dim='bin').T
                self.pmeshs[i].set_array(data.values.ravel())                   
 
        

    def interactive(self):
        """
        Creates an interactive plot with sliders according to the plot mode.
        """
        if self.mode in ['profile', 'timeseries']:
            sliders = {
                'timestep': widgets.IntSlider(min=0, max=len(self.time)-1, step=1, value=0) if self.mode == 'profile' else fixed(False),
                'x': widgets.IntSlider(min=0, max=len(self.x)-1, step=1, value=12), 
                'y': widgets.IntSlider(min=0, max=len(self.y)-1, step=1, value=12), 
                'hmin': fixed(False),
                'hmax': fixed(False),
                'savefig': fixed(False)
            }    
        elif self.mode == 'area':
            if self.timeframe == 'single':
                timestep_slider = widgets.IntSlider(min=0, max=len(self.time)-1, step=1, value=0)
            elif self.timeframe == 'interval':
                timestep_slider = widgets.IntRangeSlider(
                    value=self.timestep,
                    min=0,
                    max=len(self.time),
                    step=1,
                    description='Interval',
                    disabled=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d',
                )
            elif self.timeframe == 'all':
                timestep_slider = fixed('all')

            sliders = {
                'timestep': timestep_slider,
                'x': fixed(False),
                'y': fixed(False),
                'hmin': widgets.IntSlider(min=0, max=len(self.height)-1, step=1, value=42),
                'hmax': widgets.IntSlider(min=0, max=len(self.height)-1, step=1, value=44),
                'savefig': fixed(False),
            }
        else:
            sliders = {}
        
        interact(self.display, **sliders)
        return self


    def save_figure(self, filename: str) -> None:
        """
        Saves the figure to a file.

        Args:
            filename (str): The file name.
        """
        self.fig.savefig(filename, facecolor='white', dpi=300)




/tmp/ipykernel_2330793/2264596340.py:117: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  self.fig.subplots_adjust(hspace=0.3, wspace=0.15, left=0.15, right=0.8, top=0.85, bottom=0.15)  # make space for colorbar


[300, 400]


interactive(children=(IntRangeSlider(value=(300, 400), description='Interval', max=721), IntSlider(value=42, d…

# List Available COSMO-SPECS NetCDF Output Files

In [14]:
plots_path = '/work/bb1262/user/schimmel/cosmo-specs-torch/python/plots'
data_path = '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/' # fabians project
#data_path = '/work/bb1178/schimmel/COSMO-SPECS/testcases/RUN_ERISWILL_TESTCASE01/' # roland project

date_str = '2023020809'


file_paths = glob.glob( data_path + '*.nc' )

m_files = sorted([path for path in file_paths if path.rsplit('/', 1)[-1].startswith("M_")])
other_files = sorted([path for path in file_paths if not path.rsplit('/', 1)[-1].startswith("M_")])

sorted(file_paths)



['/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_1691768584.nc',
 '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_1691768630.nc',
 '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_1691768676.nc',
 '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_1691768714.nc',
 '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_1691768968.nc',
 '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_1691769061.nc',
 '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_1691769153.nc',
 '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_1691769187.nc',
 '/work/bb1262/user/schimmel/cosmo-specs-torch/cosmo-specs/testcases/RUN_ERISWILL_TESTCASE01/3D_16917693

# Read NetCDF Files

In [15]:
# Initialize an empty dictionary to store the datasets
data = {}

# Load each file into a separate xarray Dataset and store in the dictionary
for file in other_files:# + other_files:
    # Use the file name as the key
    key = file.rsplit('/', 1)[-1][:-3]
    
    # Open the dataset and assign to the dictionary
    data[key] = xr.open_mfdataset(file, preprocess=td.add_metadata)

list(data.keys())


['3D_1691768584',
 '3D_1691768630',
 '3D_1691768676',
 '3D_1691768714',
 '3D_1691768968',
 '3D_1691769061',
 '3D_1691769153',
 '3D_1691769187',
 '3D_1691769310']

In [ ]:
data['3D_1691768584']['hhl']

# time: 182  z: 50  y: 26  x: 36  bin: 66


<xarray.Dataset>
Dimensions:  (time: 721, z: 50, y: 26, x: 36, bin: 66)
Dimensions without coordinates: time, z, y, x, bin
Data variables: (12/25)
    t        (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>
    p0       (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>
    pp       (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>
    qv       (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>
    rho0     (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>
    rho      (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>
    ...       ...
    qf       (time, z, y, x, bin) float64 dask.array<chunksize=(721, 50, 26, 36, 66), meta=np.ndarray>
    qfw      (time, z, y, x, bin) float64 dask.array<chunksize=(721, 50, 26, 36, 66), meta=np.ndarray>
    vtf      (time, z, y, x, bin) float64 dask.array<chunksize=(721, 50, 26, 36, 66), meta=np.ndarray>
    vti      (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>
    blocks   (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>
    weight   (time, z, y, x) float64 dask.array<chunksize=(721, 50, 26, 36), meta=np.ndarray>

# Table of available COMSO-SPECS runs

In [37]:
import pandas as pd

def create_table(string_list):
    flare_types = {
        "Flare Emission (1e7)": "8.0e7",
        "Flare Emission (1e8)": "8.0e8",
        "Flare Emission (1e9)": "8.0e9",
        "No Flare Emission": "no"
    }
    
    data_dict = {key: [] for key in flare_types.keys()}
    inp_values = ["1000", "100", "10"]
    
    for inp in inp_values:
        for flare, code in flare_types.items():
            for item in string_list:
                if f"BINP{inp}_FE{code}" == item:
                    data_dict[flare].append(item[4:])
                    
    df = pd.DataFrame(data_dict, index=inp_values)
    df.index.name = "INP"
    return df


#df = create_table(data.keys())
#df.index.name = "INP"
#print(df.to_string(line_width=1000))

# Plot a horizontal slice of the entire domain

In [36]:
plot3 = MultiPanelPlot(
    data, 
    'nf', 
    nrows=3, ncols=4, 
    mode='area', 
    vmin=1.0, vmax=1.0e10, 
    timeframe='interval', # 'all', 'single'
    timestep0=[300, 400],
    ).interactive()

/tmp/ipykernel_2330793/2264596340.py:117: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  self.fig.subplots_adjust(hspace=0.3, wspace=0.15, left=0.15, right=0.8, top=0.85, bottom=0.15)  # make space for colorbar


[300, 400]


interactive(children=(IntRangeSlider(value=(300, 400), description='Interval', max=721), IntSlider(value=42, d…

# Plot a single profile for a specific location

In [14]:
plot4 = MultiPanelPlot(
    data, 
    'nf', 
    nrows=1, ncols=1, 
    mode='profile',
    ymin=0, ymax=12,
    vmin=1, vmax=1.0e6,
    ).interactive()

/tmp/ipykernel_1056776/542750403.py:117: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  self.fig.subplots_adjust(hspace=0.3, wspace=0.15, left=0.15, right=0.8, top=0.85, bottom=0.15)  # make space for colorbar


interactive(children=(IntSlider(value=0, description='timestep', max=720), IntSlider(value=12, description='x'…

In [15]:
plot4 = MultiPanelPlot(
    data, 
    'qw', 
    nrows=1, ncols=1, 
    mode='profile',
    ymin=0, ymax=12,
    vmin=1e-12, vmax=1e-6,
    ).interactive()

/tmp/ipykernel_1056776/542750403.py:117: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  self.fig.subplots_adjust(hspace=0.3, wspace=0.15, left=0.15, right=0.8, top=0.85, bottom=0.15)  # make space for colorbar


interactive(children=(IntSlider(value=0, description='timestep', max=720), IntSlider(value=12, description='x'…

In [17]:
plot4 = MultiPanelPlot(
    data, 
    'nf', 
    nrows=2, ncols=2, 
    mode='profile',
    ymin=0, ymax=2,
    vmin=1, vmax=1.0e6,
    ).interactive()

/tmp/ipykernel_1056776/542750403.py:117: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  self.fig.subplots_adjust(hspace=0.3, wspace=0.15, left=0.15, right=0.8, top=0.85, bottom=0.15)  # make space for colorbar


interactive(children=(IntSlider(value=0, description='timestep', max=720), IntSlider(value=12, description='x'…

# Plot a time series of a specific location

In [18]:
plot5 = MultiPanelPlot(
    data, 
    'nf', 
    nrows=1, ncols=1, 
    mode='timeseries',
    ymin=0, ymax=2
    ).interactive()

/tmp/ipykernel_1056776/542750403.py:117: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  self.fig.subplots_adjust(hspace=0.3, wspace=0.15, left=0.15, right=0.8, top=0.85, bottom=0.15)  # make space for colorbar


interactive(children=(IntSlider(value=12, description='x', max=35), IntSlider(value=12, description='y', max=2…